# Gas Turbine CO and NOx Emission Data Set Analysis

## Introduction

This notebook presents an initial data analysis, visualization of the data, and a Principal Component Analysis (PCA) of the **Gas Turbine CO and NOx Emission Data Set**. The dataset contains sensor measurements collected from a gas turbine, including environmental conditions, turbine operating parameters, and emissions data for **Carbon Monoxide (CO)** and **Nitrogen Oxides (NOx)**. The goal of this analysis is to better understand the dataset, uncover important relationships, and reduce dimensionality using PCA to identify key contributing features.

The dataset is sourced from the **UCI Machine Learning Repository** and can be accessed [here](https://archive.ics.uci.edu/dataset/551/gas+turbine+co+and+nox+emission+data+set).

## Data Loading and Exploration

The first step in this analysis involves loading the dataset and performing basic exploratory data analysis (EDA). This includes:
- Displaying the first few rows of the dataset to understand its structure.
- Descriptive statistics to gain insights into the data distributions (mean, median, standard deviation, etc.).
- Identification of missing values and potential data preprocessing needs.

## Data Visualization

To gain a better understanding of the data, visualizations were generated to explore the distributions and relationships between different attributes:
- **Distribution Plots**: Histograms were created to visualize the distributions of the key attributes, including ambient temperature, pressure, humidity, and the emissions of CO and NOx. These plots help identify the general behavior of the data and detect any skewness or outliers.
- **Correlation Matrix**: A correlation matrix was plotted to identify linear relationships between attributes and to highlight the variables that are highly correlated with the target emissions. This is useful for understanding which factors have the most influence on CO and NOx emissions.

## Principal Component Analysis (PCA)

Principal Component Analysis (PCA) was applied to the dataset to reduce dimensionality while retaining most of the variance:
- **Standardization**: The dataset was standardized to have a mean of zero and a standard deviation of one, ensuring comparability across attributes with different scales.
- **Explained Variance**: The explained variance plot shows that the first few principal components capture a significant portion of the total variance, allowing for a reduced representation of the data.
- **PCA Coefficients**: The contribution of each original attribute to the principal components was analyzed to understand which features are most important in explaining the variance in the data.

## Conclusion

The initial data analysis, visualization, and PCA have provided important insights into the dataset:
- The environmental conditions and turbine operating parameters have been shown to influence the emission levels of CO and NOx significantly.
- PCA has allowed us to reduce the number of features, making further analysis and model building more efficient.

These findings lay the foundation for future steps in building predictive models to estimate CO and NOx emissions based on the turbine's operating conditions.

---

In [1]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from ucimlrepo import fetch_ucirepo 

import plotly.io as pio

pio.renderers.default = "notebook"  # oder "none" für keine automatische Ausgabe

### Get, describe, and visualize data

In [2]:
# fetch dataset 
gas_turbine_co_and_nox_emission_data_set = fetch_ucirepo(id=551) 
  
# data (as pandas dataframes) 
X = gas_turbine_co_and_nox_emission_data_set.data.features 
y = gas_turbine_co_and_nox_emission_data_set.data.targets 

all = X.copy()

In [3]:
y = X[X.columns[-2:]]
X.drop(columns=X.columns[-2:], inplace=True)
X.drop(columns="year", inplace=True)

In [ ]:
desc = X.describe().round(2)
unique_values = X.nunique().round(0)
desc.loc['unique'] = unique_values
desc

In [ ]:
desc = y.describe().round(2)
unique_values = y.nunique().round(0)
desc.loc['unique'] = unique_values
desc

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

correlation_matrix = X.corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Matrix')
plt.show()

In [ ]:
#correlation without year

correlation_matrix = all.drop(columns="year").corr()

plt.figure(figsize=(10, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f", linewidths=0.5)
plt.title('Correlation Matrix')

### Distribution and scatter plots

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools  # Um alle Kombinationen der Attribute zu erstellen

# Definiere die vollständigen Variablennamen, Abkürzungen und Einheiten
full_variable_names = {
    'AT': 'Ambient temperature',
    'AP': 'Ambient pressure',
    'AH': 'Ambient humidity',
    'AFDP': 'Air filter difference pressure',
    'GTEP': 'Gas turbine exhaust pressure',
    'TIT': 'Turbine inlet temperature',
    'TAT': 'Turbine after temperature',
    'CDP': 'Compressor discharge pressure',
    'TEY': 'Turbine energy yield',
    #'CO': 'Carbon monoxide',
    #'NOx': 'Nitrogen oxides'
}

units = {
    'AT': 'C',
    'AP': 'mbar',
    'AH': '%',
    'AFDP': 'mbar',
    'GTEP': 'mbar',
    'TIT': 'C',
    'TAT': 'C',
    'CDP': 'mbar',
    'TEY': 'MWH',
    #'CO': 'mg/m3',
    #'NOx': 'mg/m3'
}

# Anzahl der Spalten und Zeilen für den Subplot
rows = 3
cols = 3

# Erstelle alle Kombinationen der Attribute
combinations = list(itertools.combinations(full_variable_names.keys(), 2))

# Erstelle den Subplot-Raster mit den Namen als Titel
fig = make_subplots(rows=rows, cols=cols, 
                    subplot_titles=[f'{full_variable_names[x]} vs {full_variable_names[y]}' 
                                    for x, y in combinations[:rows * cols]],
                    vertical_spacing=0.11)

# Iteriere durch die Attributkombinationen und füge jeden Scatterplot zu einem Subplot hinzu
current_row = 1
current_col = 1
for i, (x_attr, y_attr) in enumerate(combinations):
    if i >= rows * cols:
        break
    # Erstelle den Scatterplot
    fig.add_trace(
        go.Scatter(x=X[x_attr], y=X[y_attr], mode='markers', name=f'{x_attr} vs {y_attr}'),
        row=current_row, col=current_col
    )
    
    # Setze die Achsenbeschriftungen mit den Einheiten
    fig.update_xaxes(title_text=f'{x_attr} [{units[x_attr]}]', row=current_row, col=current_col)
    fig.update_yaxes(title_text=f'{y_attr} [{units[y_attr]}]', row=current_row, col=current_col)

    # Bewege dich zu den nächsten Spalten und Zeilen
    current_col += 1
    if current_col > cols:
        current_col = 1
        current_row += 1

# Update Layout für kleinere Titelgröße und bessere Platzierung
fig.update_layout(
    height=800, width=1000,
    title_text="Attribute Comparison Scatter Plots",
    title_x=0.5,
    showlegend=False,
    template="plotly_white",
    font=dict(size=14),  # Schriftgröße der Subplot-Titel anpassen
    annotations=[dict(font=dict(size=16)) for _ in fig.layout.annotations],  # Schriftgröße der Subplot-Titel anpassen
    margin=dict(t=80),  # Abstand für den Haupttitel
)

# Zeige das Diagramm
fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Definiere die vollständigen Variablennamen, Abkürzungen und Einheiten
full_variable_names = {
    'AT': 'Ambient temperature',
    'AP': 'Ambient pressure',
    'AH': 'Ambient humidity',
    'AFDP': 'Air filter difference pressure',
    'GTEP': 'Gas turbine exhaust pressure',
    'TIT': 'Turbine inlet temperature',
    'TAT': 'Turbine after temperature',
    'CDP': 'Compressor discharge pressure',
    'TEY': 'Turbine energy yield',
    'CO': 'Carbon monoxide',
    'NOx': 'Nitrogen oxides'
}

units = {
    'AT': 'C',
    'AP': 'mbar',
    'AH': '%',
    'AFDP': 'mbar',
    'GTEP': 'mbar',
    'TIT': 'C',
    'TAT': 'C',
    'CDP': 'mbar',
    'TEY': 'MWH',
    'CO': 'mg/m3',
    'NOx': 'mg/m3'
}

# Anzahl der Spalten und Zeilen für den Subplot
rows = 3
cols = 3

# Erstelle den Subplot-Raster mit den vollständigen Namen als Titel
fig = make_subplots(rows=rows, cols=cols, 
                    subplot_titles=[f'{full_variable_names[abbr]}' for abbr in list(full_variable_names.keys())[:rows * cols]],
                    vertical_spacing=0.11)

# Iteriere durch die Variablen und füge jedes Histogramm zu einem Subplot hinzu
current_row = 1
current_col = 1
for i, (abbr, full_name) in enumerate(full_variable_names.items()):
    if i >= rows * cols:
        break
    # Erstelle das Histogramm
    fig.add_trace(
        go.Histogram(x=X[abbr], nbinsx=30, name=full_name),
        row=current_row, col=current_col
    )
    
    # Setze die Achsenbeschriftungen mit den Einheiten (nur für col=1 die Y-Achsenbeschriftung)
    fig.update_xaxes(title_text=f'{abbr} [{units[abbr]}]', row=current_row, col=current_col)
    
    if current_col == 1:
        fig.update_yaxes(title_text='Frequency', row=current_row, col=current_col)

    # Bewege dich zu den nächsten Spalten und Zeilen
    current_col += 1
    if current_col > cols:
        current_col = 1
        current_row += 1

# Update Layout für kleinere Titelgröße und bessere Platzierung
fig.update_layout(
    height=800, width=900,
    title_text="Distribution of Attributes with Units",
    title_x=0.5,
    showlegend=False,
    template='plotly_white',
    font=dict(size=14),  # Schriftgröße der Subplot-Titel anpassen
    annotations=[dict(font=dict(size=16)) for _ in fig.layout.annotations],  # Schriftgröße der Subplot-Titel anpassen
    margin=dict(t=80),  # Abstand für den Haupttitel
)

# Zeige das Diagramm
fig.show(config={'toImageButtonOptions': {'scale': 2}})

In [ ]:
rows = 3
cols = 3

fig = make_subplots(rows=rows, cols=cols, 
                    subplot_titles=[f'{full_variable_names[abbr]}' for abbr in list(full_variable_names.keys())[:rows * cols]],
                    vertical_spacing=0.15,  # Erhöhe den vertikalen Abstand minimal
                    horizontal_spacing=0.1  # Erhöhe den horizontalen Abstand minimal
                    )

current_row = 1
current_col = 1
for i, (abbr, full_name) in enumerate(full_variable_names.items()):
    if i >= rows * cols:
        break
    # Erstelle das Histogramm
    fig.add_trace(
        go.Histogram(x=X[abbr], nbinsx=30, name=full_name),
        row=current_row, col=current_col
    )
    
    # Setze die Achsenbeschriftungen mit den Einheiten (nur für col=1 die Y-Achsenbeschriftung)
    fig.update_xaxes(title_text=f'{abbr} [{units[abbr]}]', title_font=dict(size=16), tickfont=dict(size=14), row=current_row, col=current_col)
    
    if current_col == 1:
        fig.update_yaxes(title_text='Frequency', title_font=dict(size=16), tickfont=dict(size=14), row=current_row, col=current_col)

    # Bewege dich zu den nächsten Spalten und Zeilen
    current_col += 1
    if current_col > cols:
        current_col = 1
        current_row += 1

# Update Layout für einheitliche Schriftgrößen
fig.update_layout(
    height=800, width=900,
    title_text="Distribution of Attributes with Units",
    title_x=0.5,
    showlegend=False,
    template='plotly_white',
    font=dict(size=16),  # Schriftgröße für allgemeine Texte im Layout
    annotations=[dict(font=dict(size=18)) for _ in fig.layout.annotations],  # Schriftgröße der Subplot-Titel
    margin=dict(t=80),  # Abstand für den Haupttitel
)

# Zeige das Diagramm
fig.show(config={'toImageButtonOptions': {'scale': 2}})

In [ ]:
# Erstelle einen Subplot mit 2 Zeilen und 1 Spalte für die Zielvariablen
fig_target = make_subplots(rows=2, cols=1, 
                           subplot_titles=['Carbon Monoxide', 'Nitrogen Oxides'], 
                           vertical_spacing=0.15)  # Erhöhe den vertikalen Abstand minimal, um Konsistenz sicherzustellen

# Füge das Histogramm für die erste Zielvariable hinzu
fig_target.add_trace(
    go.Histogram(x=y["CO"], nbinsx=30, name='Carbon Monoxide'),
    row=1, col=1
)

# Füge das Histogramm für die zweite Zielvariable hinzu
fig_target.add_trace(
    go.Histogram(x=y["NOX"], nbinsx=30, name='Nitrogen Oxides'),
    row=2, col=1
)

# Setze die Achsenbeschriftungen für beide Zielvariablen
fig_target.update_xaxes(title_text='CO [mg/m3]', title_font=dict(size=16), tickfont=dict(size=14), row=1, col=1)
fig_target.update_yaxes(title_text='Frequency', title_font=dict(size=16), tickfont=dict(size=14), row=1, col=1)

fig_target.update_xaxes(title_text='NOx [mg/m3]', title_font=dict(size=16), tickfont=dict(size=14), row=2, col=1)
fig_target.update_yaxes(title_text='Frequency', title_font=dict(size=16), tickfont=dict(size=14), row=2, col=1)

# Update Layout für einheitliche Schriftgrößen und gleiche Größen wie der Attributplot
fig_target.update_layout(
    height=800, width=450,  # Höhe und Breite anpassen, um den Plot neben den Attributplot zu stellen
    title_text="Distribution of Target Variables",
    title_x=0.5,
    showlegend=False,
    template='plotly_white',
    font=dict(size=16),  # Schriftgröße für allgemeine Texte im Layout
    annotations=[dict(font=dict(size=18)) for _ in fig_target.layout.annotations],  # Schriftgröße der Subplot-Titel
    margin=dict(t=80),  # Abstand für den Haupttitel
)

# Zeige den Subplot für die zwei Zielvariablen
fig_target.show(config={'toImageButtonOptions': {'scale': 2}})

In [ ]:
# Erstelle einen Subplot mit 1 Zeile und 2 Spalten für die Zielvariablen
fig = make_subplots(rows=1, cols=2, subplot_titles=['Carbon Monoxide', 'Nitrogen oxides'])

# Füge das Histogramm für die erste Zielvariable hinzu
fig.add_trace(
    go.Histogram(x=y["CO"], nbinsx=1000, name='Target Variable 1'),
    row=1, col=1
)

# Füge das Histogramm für die zweite Zielvariable hinzu
fig.add_trace(
    go.Histogram(x=y["NOX"], nbinsx=30, name='Nitrogen oxides'),
    row=1, col=2
)

# Setze die Achsenbeschriftungen für beide Zielvariablen
fig.update_xaxes(title_text='CO [mg/m3]', row=1, col=1)
fig.update_yaxes(title_text='Frequency', row=1, col=1)

fig.update_xaxes(title_text='NOx [mg/m3]', row=1, col=2)
#fig.update_yaxes(title_text='Frequency', row=1, col=2)

# Update Layout
fig.update_layout(
    height=400, width=800,
    title_text="Distribution of Target Variables",
    title_x=0.5,
    showlegend=False,
    template='plotly_white',
    #annotations=[dict(font=dict(size=12)) for _ in fig.layout.annotations],  # Schriftgröße der Subplot-Titel anpassen
)

# Zeige den Subplot für die zwei Zielvariablen
fig.show()

In [ ]:
import plotly.graph_objects as go

# Erstelle ein separates Histogramm für den Bereich 15-50 mg/m³ der CO-Werte
fig = go.Figure()

# Füge das Histogramm für den Bereich 15-50 mg/m³ hinzu
fig.add_trace(
    go.Histogram(x=[val for val in y["CO"] if 15 <= val <= 50], nbinsx=50, marker_color='blue')
)

# Setze die Achsen ohne Titel und Achsenbeschriftungen
# fig.update_xaxes(showticklabels=False)
# fig.update_yaxes(showticklabels=False)

# Füge einen gestrichelten Rahmen außerhalb der Achsenbeschriftungen hinzu
fig.update_layout(
    height=400, width=600,
    showlegend=False,
    template='plotly_white',
    shapes=[
        # Rechteckiger Rahmen außerhalb der Achsenbeschriftungen
        dict(
            type="rect",
            xref="paper", yref="paper",
            x0=-0.1, y0=-0.1, x1=1.1, y1=1.1,  # Vergrößere die Grenzen des Rahmens
            line=dict(
                color="black",
                width=2,
                dash="dash"  # gestrichelte Linie
            )
        )
    ]
)

# Zeige den separaten Barplot
fig.show()

In [ ]:
import plotly.graph_objects as go

# Erstelle ein separates Histogramm für den Bereich 15-50 mg/m³ der CO-Werte
fig = go.Figure()

# Füge das Histogramm für den Bereich 15-50 mg/m³ hinzu
fig.add_trace(
    go.Histogram(x=[val for val in y["CO"] if 15 <= val <= 50], nbinsx=50, marker_color='blue')
)

# Setze die Achsen mit abgestimmten Schriftgrößen und höher aufgelösten Ticks auf der x-Achse
fig.update_xaxes(
    #title_text='CO [mg/m3]',
    title_font={"size": 16},  # Schriftgröße für den Titel der X-Achse
    tickfont={"size": 14},
    tick0=15,  # Startpunkt der Ticks
    dtick=5  # Abstand zwischen den Ticks
)
fig.update_yaxes(
    #title_text='Frequency',
    title_font={"size": 16},  # Schriftgröße für den Titel der Y-Achse
    tickfont={"size": 14}
)

# Füge einen gestrichelten Rahmen außerhalb der Achsenbeschriftungen hinzu
fig.update_layout(
    height=300, width=350,  # Leicht erhöhte Größe, damit der Rahmen nicht abgeschnitten wird
    showlegend=False,
    template='plotly_white',
    shapes=[
        # Rechteckiger Rahmen außerhalb der Achsenbeschriftungen
        dict(
            type="rect",
            xref="paper", yref="paper",
            x0=-0.15, y0=-0.15, x1=1.15, y1=1.15,  # Vergrößere die Grenzen des Rahmens weiter nach außen
            line=dict(
                color="black",
                width=2,
                dash="dash"  # gestrichelte Linie
            )
        )
    ],
    margin=dict(l=50, r=50, t=50, b=50)  # Größere Ränder, um Platz für den Rahmen zu lassen
)

# Zeige den separaten Barplot
fig.show()

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Erstelle einen Subplot mit 1 Zeile und 2 Spalten für die Zielvariablen
fig = make_subplots(rows=1, cols=2, subplot_titles=['Carbon Monoxide', 'Nitrogen oxides'])

# Füge das Histogramm für die erste Zielvariable hinzu (Carbon Monoxide)
fig.add_trace(
    go.Histogram(x=y["CO"], nbinsx=1000, name='Carbon Monoxide', marker_color='blue'),
    row=1, col=1
)

# Füge das Histogramm für die zweite Zielvariable hinzu (Nitrogen oxides)
fig.add_trace(
    go.Histogram(x=y["NOX"], nbinsx=30, name='Nitrogen oxides', marker_color='red'),
    row=1, col=2
)

# Setze die Achsenbeschriftungen für beide Zielvariablen
fig.update_xaxes(title_text='CO [mg/m3]', row=1, col=1)
fig.update_yaxes(title_text='Frequency', row=1, col=1)

fig.update_xaxes(title_text='NOx [mg/m3]', row=1, col=2)

# Erstelle den Insert-Plot für den Bereich 10-50 mg/m³ von CO
trace_inset = go.Histogram(
    x=[val for val in y["CO"] if 10 <= val <= 50], 
    nbinsx=30, 
    marker_color='blue', 
    showlegend=False
)

# Füge den Insert Plot als eigenständigen Bereich (x2, y2) hinzu
fig.add_trace(trace_inset)

# Update Layout, um den Insert Plot eigenständig zu positionieren
fig.update_layout(
    xaxis3=dict(
        domain=[0.3, 0.6],  # Horizontaler Bereich für den Insert Plot innerhalb des ersten Subplots (CO)
        anchor='y3',
        title_text='CO [mg/m3] (Zoomed 10-50 mg/m³)',
    ),
    yaxis3=dict(
        domain=[0.5, 0.9],  # Vertikaler Bereich für den Insert Plot
        anchor='x3',
        title_text='Frequency'
    ),
    height=400, 
    width=800,
    title_text="Distribution of Target Variables",
    title_x=0.5,
    showlegend=False,
    template='plotly_white'
)

# Zeige den Subplot mit dem eigenständigen Insert-Plot
fig.show()

### Standardized PCA

In [13]:
# Standardisiere die Daten (Zero-Mean-Normalisierung)
scaler = StandardScaler()
scaled_data = scaler.fit_transform(X)

# Führe PCA durch
pca = PCA(n_components=len(X.columns))
pca.fit(scaled_data)

# Variance explained ratios
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

In [ ]:
import plotly.graph_objects as go

# Erstelle den Plotly-Plot
fig = go.Figure()

# Hinzufügen der Einzelvarianz (Individual) als Linie mit Punkten
fig.add_trace(go.Scatter(x=list(range(1, len(explained_variance_ratio) + 1)),
                         y=explained_variance_ratio,
                         mode='lines+markers',
                         name='Individual Variance',
                         marker=dict(symbol='x'),
                         line=dict(shape='linear')))

# Hinzufügen der kumulierten Varianz (Cumulative) als Linie mit Punkten
fig.add_trace(go.Scatter(x=list(range(1, len(cumulative_variance_ratio) + 1)),
                         y=cumulative_variance_ratio,
                         mode='lines+markers',
                         name='Cumulative Variance',
                         marker=dict(symbol='circle'),
                         line=dict(shape='linear')))

# Schwellenlinien für 90%
fig.add_trace(go.Scatter(x=[1, len(explained_variance_ratio)], y=[0.9, 0.9],
                         mode='lines', name='Threshold 90%', line=dict(dash='dash', color='black')))

# Layout für den Plot mit abgestimmten Schriftgrößen
fig.update_layout(
    title='Variance explained by Principal Components',
    title_x=0.5,
    xaxis_title='Principal component',
    yaxis_title='Variance explained',
    legend_title='Legend',
    legend=dict(
        x=0.7,
        y=0.5,
        font=dict(size=14)
    ),
    template='plotly_white',
    width=800,
    height=500,
    font=dict(size=16),  # Schriftgröße für allgemeine Texte im Layout
    xaxis=dict(
        title_font=dict(size=16),  # Schriftgröße für den Titel der X-Achse
        tickfont=dict(size=14)     # Schriftgröße für die Werte der X-Achse
    ),
    yaxis=dict(
        title_font=dict(size=16),  # Schriftgröße für den Titel der Y-Achse
        tickfont=dict(size=14)     # Schriftgröße für die Werte der Y-Achse
    )
)

# Plot anzeigen
fig.show(config={'toImageButtonOptions': {'scale': 2}})

In [ ]:
import plotly.graph_objects as go
import numpy as np

# Assuming you have already defined X, StandardScaler, and PCA
# attrnames: The names of the attributes (this corresponds to the column names of your dataset)
attrnames = X.columns.tolist()  # Attribute names

# The PCA components matrix (V corresponds to pca.components_.T because you need the transposed eigenvectors)
V = pca.components_.T

# Principal components to plot
pcs = [0, 1, 2, 3]  # Adjust this if you want to plot more or fewer components

# Labels for the legend
legendStrs = ["PC" + str(e + 1) for e in pcs]

# Bar width and positions for attributes
bw = 0.2  # Bar width
r = np.arange(1, len(attrnames) + 1)  # X-axis positions for attributes

# Create an empty figure
fig = go.Figure()

# Add bars for each principal component
for i, pc in enumerate(pcs):
    fig.add_trace(go.Bar(
        x=r + i * bw,
        y=V[:, pc],  # The coefficients for the current principal component
        name=legendStrs[i],
        width=bw,
        marker_line_color='black',  # Black border
        marker_line_width=0.5  # Thickness of the border
    ))

# Update layout to customize the plot with consistent label sizes
fig.update_layout(
    title="Principal Component Coefficients for Each Attribute",
    title_x=0.5,
    xaxis_title="Attributes",
    yaxis_title="Component coefficients",
    xaxis=dict(
        tickvals=r + bw,  # Set tick positions
        ticktext=attrnames,  # Set the attribute names as tick labels
        title_font=dict(size=16),  # Schriftgröße für den Titel der X-Achse
        tickfont=dict(size=14)  # Schriftgröße für die Werte der X-Achse
    ),
    yaxis=dict(
        title_font=dict(size=16),  # Schriftgröße für den Titel der Y-Achse
        tickfont=dict(size=14)  # Schriftgröße für die Werte der Y-Achse
    ),
    legend_title="Principal Components",
    legend=dict(
        x=0.75,
        y=0.05,
        font=dict(size=14)  # Schriftgröße für die Legende
    ),
    template="plotly_white",
    barmode='overlay',
    width=800,
    height=500,
    font=dict(size=16)  # Schriftgröße für allgemeine Texte im Layout
)

# Show the figure
fig.show(config={'toImageButtonOptions': {'scale': 2}})

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Erstelle einen DataFrame für die ersten 4 PCA-Komponenten
pca_data = pca.fit_transform(scaled_data)
pca_df = pd.DataFrame(pca_data[:, :4], columns=[f'PC{i+1}' for i in range(4)])

# Anzahl der Hauptkomponenten
n_components = 4

# Zielvariable NOx (verwende dies als Farbindikator)
nox = y["NOX"]

# Erstelle ein Subplot-Grid mit separaten Achsenwie du
fig = make_subplots(rows=n_components, cols=n_components, 
                    shared_xaxes=False, shared_yaxes=False,
                    horizontal_spacing=0.05, vertical_spacing=0.05)

# Invertierte Farbskala (niedrig = hell, hoch = dunkel)
inverted_colorscale = 'Viridis_r'  # Das "_r" am Ende kehrt die Farbskala um

# Iteriere durch die Hauptkomponenten, um die Plots zu erstellen
for i in range(n_components):
    for j in range(n_components):
        if i == j:
            # Diagonale: Histogramme der Verteilung der Hauptkomponente in Schwarz
            fig.add_trace(go.Histogram(x=pca_df.iloc[:, i], showlegend=False,
                                       marker=dict(color='black', line=dict(width=0))),  # Schwarz ohne Umrandung
                          row=i+1, col=j+1)
        else:
            # Off-Diagonale: Scatterplots mit umgekehrtem Colorgrading basierend auf NOx
            fig.add_trace(go.Scatter(x=pca_df.iloc[:, j], y=pca_df.iloc[:, i],
                                     mode='markers',
                                     marker=dict(size=6, color=nox, colorscale=inverted_colorscale,  # Umgekehrte Farbskala
                                                 #line=dict(color='black', width=1),
                                                 coloraxis="coloraxis"),
                                     showlegend=False),
                          row=i+1, col=j+1)
            
        # Beschrifte nur die äußeren Achsen und stelle sicher, dass der Abstand gleich ist
        if i == n_components - 1:
            fig.update_xaxes(title_text=f'PC{j+1}', row=i+1, col=j+1, title_standoff=5, automargin=True)
        if j == 0:
            fig.update_yaxes(title_text=f'PC{i+1}', row=i+1, col=j+1, title_standoff=5, automargin=True)

# Aktualisiere Layout und füge die Farbskala für NOx hinzu
fig.update_layout(
    title="Pairwise Plot of First 4 PCs with NOx Gradient",
    title_x=0.5,
    width=800, height=800,
    template='plotly_white',
    yaxis=dict(automargin=True),  # Aktiviert automatisches Anpassen der Achsenbeschriftungen
    coloraxis=dict(colorscale=inverted_colorscale, colorbar=dict(title="NOx [mg/m3]"))  # Hinzufügen der Farbskala zur Legende
)

# Zeige den Plot
fig.show(config={'toImageButtonOptions': {'scale': 2}})

In [ ]:
import plotly.io as pio
from IPython.display import Image

# Speichere das Bild als PNG auf deinem lokalen System
pio.write_image(fig, "pca_pairplot.png", format='png')

# Lade das Bild und zeige es an
Image("pca_pairplot.png")

### Non-Standardized PCA

In [8]:
# Führe PCA ohne Standardisierung durch
pca = PCA(n_components=len(X.columns))
pca.fit(X)  # Verwende die Originaldaten ohne Skalierung

# Variance explained ratios
explained_variance_ratio = pca.explained_variance_ratio_
cumulative_variance_ratio = np.cumsum(explained_variance_ratio)

In [ ]:
# Erstelle den Plotly-Plot
fig = go.Figure()

# Hinzufügen der Einzelvarianz (Individual) als Linie mit Punkten
fig.add_trace(go.Scatter(x=list(range(1, len(explained_variance_ratio) + 1)),
                         y=explained_variance_ratio,
                         mode='lines+markers',
                         name='Individual Variance',
                         marker=dict(symbol='x'),
                         line=dict(shape='linear')))

# Hinzufügen der kumulierten Varianz (Cumulative) als Linie mit Punkten
fig.add_trace(go.Scatter(x=list(range(1, len(cumulative_variance_ratio) + 1)),
                         y=cumulative_variance_ratio,
                         mode='lines+markers',
                         name='Cumulative Variance',
                         marker=dict(symbol='circle'),
                         line=dict(shape='linear')))

# Schwellenlinien für 90%
fig.add_trace(go.Scatter(x=[1, len(explained_variance_ratio)], y=[0.9, 0.9],
                         mode='lines', name='Threshold 90%', line=dict(dash='dash', color='black')))


# Layout für den Plot
fig.update_layout(
    title='Variance explained by Principal Components',
    title_x=0.5,
    xaxis_title='Principal component',
    yaxis_title='Variance explained',
    legend_title='Legend',
    legend_x=0.7,
    legend_y=0.5,
    template='plotly_white',
    width=600
)

# Plot anzeigen
fig.show()

In [ ]:
# Erstelle einen DataFrame für die ersten 4 PCA-Komponenten
pca_data = pca.fit_transform(X)
pca_df = pd.DataFrame(pca_data[:, :4], columns=[f'PC{i+1}' for i in range(4)])

# Anzahl der Hauptkomponenten
n_components = 4

# Erstelle ein Subplot-Grid mit separaten Achsen
fig = make_subplots(rows=n_components, cols=n_components, 
                    shared_xaxes=False, shared_yaxes=False,
                    horizontal_spacing=0.05, vertical_spacing=0.05)

# Iteriere durch die Hauptkomponenten, um die Plots zu erstellen
for i in range(n_components):
    for j in range(n_components):
        if i == j:
            # Diagonale: Histogramme der Verteilung der Hauptkomponente in Schwarz
            fig.add_trace(go.Histogram(x=pca_df.iloc[:, i], showlegend=False,
                                       marker=dict(color='black', line=dict(width=0))),  # Schwarz ohne Umrandung
                          row=i+1, col=j+1)
        else:
            # Off-Diagonale: Scatterplots mit umrandeten Punkten
            fig.add_trace(go.Scatter(x=pca_df.iloc[:, j], y=pca_df.iloc[:, i],
                                     mode='markers',
                                     marker=dict(size=6, line=dict(color='black', width=1)),
                                     showlegend=False),
                          row=i+1, col=j+1)
            
        # Beschrifte nur die äußeren Achsen
        if i == n_components - 1:
            fig.update_xaxes(title_text=f'PC{j+1}', row=i+1, col=j+1)
        if j == 0:
            fig.update_yaxes(title_text=f'PC{i+1}', row=i+1, col=j+1)
            
        fig.update_yaxes(title_text=f'PC{1}', row=1, col=1, title_standoff=0)

# Aktualisiere Layout
fig.update_layout(
    title="Pairwise comparison of first 4 Principal Components",
    title_x=0.5,
    width=800, height=800,
    template='plotly_white',
    yaxis=dict(automargin=True),  # Aktiviert automatisches Anpassen der Achsenbeschriftungen
)

# Zeige den Plot
# fig.show()

In [ ]:
import plotly.io as pio
from IPython.display import Image

# Speichere das Bild als PNG auf deinem lokalen System
pio.write_image(fig, "pca_pairplot_non_standardized.png", format='png')

# Lade das Bild und zeige es an
Image("pca_pairplot.png")

### Create figures for report

In [7]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Erstelle einen DataFrame für die ersten 4 PCA-Komponenten
pca_data = pca.fit_transform(scaled_data)
pca_df = pd.DataFrame(pca_data[:, :4], columns=[f'PC{i+1}' for i in range(4)])

# Anzahl der Hauptkomponenten
n_components = 4

# Zielvariable NOx (verwende dies als Farbindikator)
nox = y["NOX"]

# Erstelle ein Subplot-Grid mit separaten Achsen
fig = make_subplots(rows=n_components, cols=n_components, 
                    shared_xaxes=False, shared_yaxes=False,
                    horizontal_spacing=0.05, vertical_spacing=0.05)

# Invertierte Farbskala (niedrig = hell, hoch = dunkel)
inverted_colorscale = 'Viridis_r'  # Das "_r" am Ende kehrt die Farbskala um

# Iteriere durch die Hauptkomponenten, um die Plots zu erstellen
for i in range(n_components):
    for j in range(n_components):
        if i == j:
            # Diagonale: Histogramme der Verteilung der Hauptkomponente in Schwarz
            fig.add_trace(go.Histogram(x=pca_df.iloc[:, i], showlegend=False,
                                       marker=dict(color='black', line=dict(width=0))),  # Schwarz ohne Umrandung
                          row=i+1, col=j+1)
        else:
            # Off-Diagonale: Scatterplots mit umgekehrtem Colorgrading basierend auf NOx
            fig.add_trace(go.Scatter(x=pca_df.iloc[:, j], y=pca_df.iloc[:, i],
                                     mode='markers',
                                     marker=dict(size=6, color=nox, colorscale=inverted_colorscale,  # Umgekehrte Farbskala
                                                 coloraxis="coloraxis"),
                                     showlegend=False),
                          row=i+1, col=j+1)
            
        # Beschrifte nur die äußeren Achsen und stelle sicher, dass der Abstand gleich ist
        if i == n_components - 1:
            fig.update_xaxes(title_text=f'PC{j+1}', row=i+1, col=j+1, title_standoff=5, automargin=True)
        if j == 0:
            fig.update_yaxes(title_text=f'PC{i+1}', row=i+1, col=j+1, title_standoff=5, automargin=True)

# Aktualisiere Layout und füge die Farbskala für NOx hinzu
fig.update_layout(
    title="Pairwise Plot of First 4 PCs with NOx Gradient",
    title_x=0.5,
    width=800, height=800,
    template='plotly_white',
    yaxis=dict(automargin=True),  # Aktiviert automatisches Anpassen der Achsenbeschriftungen
    coloraxis=dict(colorscale=inverted_colorscale, colorbar=dict(title="NOx [mg/m3]"))  # Hinzufügen der Farbskala zur Legende
)

# Speichere das Bild als PNG mit hoher Auflösung (z.B. 2x Skala)
fig.write_image("pca_nox_plot.png", scale=2)  # Speichert das Bild im aktuellen Verzeichnis

In [8]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Erstelle einen DataFrame für die ersten 4 PCA-Komponenten
pca_data = pca.fit_transform(scaled_data)
pca_df = pd.DataFrame(pca_data[:, :4], columns=[f'PC{i+1}' for i in range(4)])

# Anzahl der Hauptkomponenten
n_components = 4

# Zielvariable CO (verwende dies als Farbindikator)
co = y["CO"]  # Verwende die CO-Werte für die Farbskala

# Erstelle ein Subplot-Grid mit separaten Achsen
fig = make_subplots(rows=n_components, cols=n_components, 
                    shared_xaxes=False, shared_yaxes=False,
                    horizontal_spacing=0.05, vertical_spacing=0.05)

# Invertierte Farbskala (niedrig = hell, hoch = dunkel)
inverted_colorscale = 'Viridis_r'  # Das "_r" am Ende kehrt die Farbskala um

# Iteriere durch die Hauptkomponenten, um die Plots zu erstellen
for i in range(n_components):
    for j in range(n_components):
        if i == j:
            # Diagonale: Histogramme der Verteilung der Hauptkomponente in Schwarz
            fig.add_trace(go.Histogram(x=pca_df.iloc[:, i], showlegend=False,
                                       marker=dict(color='black', line=dict(width=0))),  # Schwarz ohne Umrandung
                          row=i+1, col=j+1)
        else:
            # Off-Diagonale: Scatterplots mit umgekehrtem Colorgrading basierend auf CO
            fig.add_trace(go.Scatter(x=pca_df.iloc[:, j], y=pca_df.iloc[:, i],
                                     mode='markers',
                                     marker=dict(size=6, color=co, colorscale=inverted_colorscale,  # Umgekehrte Farbskala
                                                 coloraxis="coloraxis"),
                                     showlegend=False),
                          row=i+1, col=j+1)
            
        # Beschrifte nur die äußeren Achsen und stelle sicher, dass der Abstand gleich ist
        if i == n_components - 1:
            fig.update_xaxes(title_text=f'PC{j+1}', row=i+1, col=j+1, title_standoff=5, automargin=True)
        if j == 0:
            fig.update_yaxes(title_text=f'PC{i+1}', row=i+1, col=j+1, title_standoff=5, automargin=True)

# Aktualisiere Layout und füge die Farbskala für CO hinzu
fig.update_layout(
    title="Pairwise Plot of First 4 PCs with CO Gradient",
    title_x=0.5,
    width=800, height=800,
    template='plotly_white',
    yaxis=dict(automargin=True),  # Aktiviert automatisches Anpassen der Achsenbeschriftungen
    coloraxis=dict(colorscale=inverted_colorscale, colorbar=dict(title="CO [mg/m3]"))  # Hinzufügen der Farbskala zur Legende
)

# Speichere das Bild als PNG mit hoher Auflösung (z.B. 2x Skala)
fig.write_image("pca_co_plot.png", scale=2)  # Speichert das Bild im aktuellen Verzeichnis

In [10]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Erstelle einen DataFrame für die ersten 4 PCA-Komponenten
pca_data = pca.fit_transform(scaled_data)
pca_df = pd.DataFrame(pca_data[:, :4], columns=[f'PC{i+1}' for i in range(4)])

# Anzahl der Hauptkomponenten
n_components = 4

# Zielvariable CO (verwende dies als Farbindikator)
co = y["CO"]  # Verwende die CO-Werte für die Farbskala

# Erstelle ein Subplot-Grid mit separaten Achsen
fig = make_subplots(rows=n_components, cols=n_components, 
                    shared_xaxes=False, shared_yaxes=False,
                    horizontal_spacing=0.05, vertical_spacing=0.05)

# Verwende das gleiche Farbschema wie bei NOx (Viridis_r)
viridis_colorscale = 'Viridis_r'

# Iteriere durch die Hauptkomponenten, um die Plots zu erstellen
for i in range(n_components):
    for j in range(n_components):
        if i == j:
            # Diagonale: Histogramme der Verteilung der Hauptkomponente in Schwarz
            fig.add_trace(go.Histogram(x=pca_df.iloc[:, i], showlegend=False,
                                       marker=dict(color='black', line=dict(width=0))),  # Schwarz ohne Umrandung
                          row=i+1, col=j+1)
        else:
            # Off-Diagonale: Scatterplots mit dem gleichen Colorgrading wie NOx
            fig.add_trace(go.Scatter(x=pca_df.iloc[:, j], y=pca_df.iloc[:, i],
                                     mode='markers',
                                     marker=dict(size=6, color=co, colorscale=viridis_colorscale,  # Verwende Viridis_r
                                                 opacity=0.8,  # Reduziere die Deckkraft für bessere Sichtbarkeit
                                                 coloraxis="coloraxis"),
                                     showlegend=False),
                          row=i+1, col=j+1)
            
        # Beschrifte nur die äußeren Achsen und stelle sicher, dass der Abstand gleich ist
        if i == n_components - 1:
            fig.update_xaxes(title_text=f'PC{j+1}', row=i+1, col=j+1, title_standoff=5, automargin=True)
        if j == 0:
            fig.update_yaxes(title_text=f'PC{i+1}', row=i+1, col=j+1, title_standoff=5, automargin=True)

# Aktualisiere Layout und füge die Farbskala für CO hinzu
fig.update_layout(
    title="Pairwise Plot of First 4 PCs with CO Gradient (Viridis_r)",
    title_x=0.5,
    width=800, height=800,
    template='plotly_white',
    yaxis=dict(automargin=True),  # Aktiviert automatisches Anpassen der Achsenbeschriftungen
    coloraxis=dict(colorscale=viridis_colorscale, colorbar=dict(title="CO [mg/m3]"))  # Hinzufügen der Farbskala zur Legende
)

# Speichere das Bild als PNG mit hoher Auflösung (z.B. 2x Skala)
fig.write_image("pca_co_viridis_plot.png", scale=2)  # Speichert das Bild im aktuellen Verzeichnis

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# Erstelle einen DataFrame für die ersten 4 PCA-Komponenten
pca_data = pca.fit_transform(scaled_data)
pca_df = pd.DataFrame(pca_data[:, :4], columns=[f'PC{i+1}' for i in range(4)])

# Anzahl der Hauptkomponenten
n_components = 4

# Zielvariable CO (verwende dies als Farbindikator)
co = y["CO"]  # Verwende die CO-Werte für die Farbskala

# Erstelle ein Subplot-Grid mit separaten Achsen
fig = make_subplots(rows=n_components, cols=n_components, 
                    shared_xaxes=False, shared_yaxes=False,
                    horizontal_spacing=0.05, vertical_spacing=0.05)

# Invertierte Farbskala basierend auf Viridis_r mit Stauchung im Bereich 1-10
stretched_viridis_r_colorscale = [
    [0, "rgb(253, 231, 37)"],   # Hellgelb für Werte nahe 0 (niedrigste CO-Werte)
    [0.25, "rgb(33, 145, 140)"],  # Heller Bereich für Werte um 5
    [0.50, "rgb(44, 114, 142)"],  # Stärkere Blau-Grün-Töne für Werte um 10
    [0.75, "rgb(59, 81, 139)"],  # Dunklere Töne für Werte um 20-30
    [1, "rgb(68, 1, 84)"]  # Dunkles Lila für die höchsten Werte (40)
]

# Iteriere durch die Hauptkomponenten, um die Plots zu erstellen
for i in range(n_components):
    for j in range(n_components):
        if i == j:
            # Diagonale: Histogramme der Verteilung der Hauptkomponente in Schwarz
            fig.add_trace(go.Histogram(x=pca_df.iloc[:, i], showlegend=False,
                                       marker=dict(color='black', line=dict(width=0))),  # Schwarz ohne Umrandung
                          row=i+1, col=j+1)
        else:
            # Off-Diagonale: Scatterplots mit der gestauchten invertierten Viridis-Farbskala basierend auf CO
            fig.add_trace(go.Scatter(x=pca_df.iloc[:, j], y=pca_df.iloc[:, i],
                                     mode='markers',
                                     marker=dict(size=6, color=co, colorscale=stretched_viridis_r_colorscale,  # Gestauchte Farbskala
                                                 opacity=0.8,  # Reduziere die Deckkraft für bessere Sichtbarkeit
                                                 coloraxis="coloraxis"),
                                     showlegend=False),
                          row=i+1, col=j+1)
            
        # Beschrifte nur die äußeren Achsen und stelle sicher, dass der Abstand gleich ist
        if i == n_components - 1:
            fig.update_xaxes(title_text=f'PC{j+1}', row=i+1, col=j+1, title_standoff=5, automargin=True)
        if j == 0:
            fig.update_yaxes(title_text=f'PC{i+1}', row=i+1, col=j+1, title_standoff=5, automargin=True)

# Aktualisiere Layout und füge die Farbskala für CO hinzu
fig.update_layout(
    title="Pairwise Plot of First 4 PCs with CO Gradient (Stretched Viridis_r)",
    title_x=0.5,
    width=800, height=800,
    template='plotly_white',
    yaxis=dict(automargin=True),  # Aktiviert automatisches Anpassen der Achsenbeschriftungen
    coloraxis=dict(colorscale=stretched_viridis_r_colorscale, colorbar=dict(title="CO [mg/m3]"))  # Hinzufügen der Farbskala zur Legende
)

# Speichere das Bild als PNG mit hoher Auflösung (z.B. 2x Skala)
fig.write_image("pca_co_stretched_viridis_r_plot.png", scale=2)  # Speichert das Bild im aktuellen Verzeichnis